# Stacked Bar Graph
## Tech Stack Count by Role

In [1]:
import pandas
from altair import Chart, Tooltip, Color, X, Y
from pandas import DataFrame

from app.data import MongoDB

### Data Source

In [2]:
db = MongoDB()

### Data Engineering

In [3]:
mentees = DataFrame(
    db.projection("Mentees", {"is_active": True, "validate_status": "approved"},
                  {"tech_stack": True}))
mentees

,tech_stack
0,Frontend
1,iOS
2,Frontend
3,iOS
4,Frontend
5,Frontend
6,iOS
7,Career Development
8,Backend
9,Design UI/UX


In [4]:
mentees["role"] = "Mentee"
mentees

,tech_stack,role
0,Frontend,Mentee
1,iOS,Mentee
2,Frontend,Mentee
3,iOS,Mentee
4,Frontend,Mentee
5,Frontend,Mentee
6,iOS,Mentee
7,Career Development,Mentee
8,Backend,Mentee
9,Design UI/UX,Mentee


In [5]:
mentors = DataFrame(db.projection("Mentors", {
    "accepting_new_mentees": True,
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True}))
mentors = mentors.explode(column="tech_stack")
mentors["role"] = "Mentor"
mentors


,tech_stack,role
0,Android,Mentor
0,Data Science,Mentor
1,iOS,Mentor
2,Design UI/UX,Mentor
3,Career Development,Mentor
4,Design UI/UX,Mentor
4,Android,Mentor
4,iOS,Mentor
5,iOS,Mentor
5,Career Development,Mentor


In [6]:
df = pandas.concat([mentees, mentors])
df.sample(10)

,tech_stack,role
6,iOS,Mentee
2,Design UI/UX,Mentor
31,Backend,Mentee
35,Frontend,Mentee
36,Frontend,Mentee
33,Frontend,Mentee
5,Career Development,Mentor
13,Frontend,Mentee
20,Backend,Mentee
9,Design UI/UX,Mentee


### Data Visualization


In [7]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")

In [8]:
def stacked_bar_chart(df, column_1, column_2):
    return Chart(
        df,
        title=f"{title_fix(column_1)} Count by {title_fix(column_2)}",
    ).mark_bar().encode(
        x=X(column_1, title=title_fix(column_1), sort="-y"),
        y=Y(f"count({column_1})"),
        color=Color(column_2, title=title_fix(column_2)),
        tooltip=Tooltip([column_2, column_1, f"count({column_1})"])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )



In [9]:
chart = stacked_bar_chart(df, "tech_stack", "role")
chart

alt.Chart(...)

In [10]:
chart.to_dict()


{'config': {'view': {'continuousWidth': 400,
   'continuousHeight': 300,
   'stroke': '#FFF'},
  'legend': {'padding': 24},
  'title': {'fontSize': 20, 'offset': 24}},
 'data': {'name': 'data-31e97a1d4edc919d478e0cd56a0f7459'},
 'mark': 'bar',
 'encoding': {'color': {'field': 'role', 'title': 'Role', 'type': 'nominal'},
  'tooltip': [{'field': 'role', 'type': 'nominal'},
   {'field': 'tech_stack', 'type': 'nominal'},
   {'aggregate': 'count', 'field': 'tech_stack', 'type': 'nominal'}],
  'x': {'field': 'tech_stack',
   'sort': '-y',
   'title': 'Tech Stack',
   'type': 'nominal'},
  'y': {'aggregate': 'count', 'field': 'tech_stack', 'type': 'nominal'}},
 'height': 400,
 'padding': 24,
 'title': 'Tech Stack Count by Role',
 'width': 480,
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.17.0.json',
 'datasets': {'data-31e97a1d4edc919d478e0cd56a0f7459': [{'tech_stack': 'Frontend',
    'role': 'Mentee'},
   {'tech_stack': 'iOS', 'role': 'Mentee'},
   {'tech_stack': 'Frontend', 'role

In [12]:
import requests
from altair import Chart

chart = requests.get("http://underdog-devs-ds-a-dev.us-east-1.elasticbeanstalk.com/graph/tech-stack-by-role").json()
Chart.from_dict(chart)

alt.Chart(...)